#Mock

Как говорилось выше, высокая связанность внутри модулей программы приводит к сложностям при написании unit-тестов. Облегчить жизнь разработчикам призван класс unittest.Mock.

Mock — это объект в тестировании, который заменяет реальный объект, имитируя его поведение.

Например, при написании тестов к системе, в которой реализованы http-запросы к реальному сервису, будет целесообразно использовать Mock-объекты как результат ответа на определенный запрос. Т.к. легко может произойти ситуация, когда тест не будет проходить в силу отсутствия соединения с сервером / вернулся ответ в другом формате и т.д.

Чтобы создать Mock-объект, нужно:

In [1]:
from unittest.mock import Mock

mock = Mock()
mock

<Mock id='140306022132880'>

Особенностью работы с Mock является создание объектов «на лету» во время первого обращения к ним:

In [2]:
mock.some_attribute

<Mock name='mock.some_attribute' id='140305959970512'>

In [3]:
mock.do_something

<Mock name='mock.do_something' id='140305959897232'>

In [4]:
mock.go

<Mock name='mock.go' id='140305960027984'>

Внутри Mock хранится история использования данного объекта.

In [20]:
from unittest.mock import Mock

json = Mock()
json.loads = ('{"key": "value"}')
json.loads.assert_called()
json.loads.assert_called_once()
json.loads.assert_called_with('{"key": "value"}')
json.loads.assert_called_once_with('{"key": "value"}')

AttributeError: ignored

__assert_called()__ — объект был вызван.

__assert_called_once()__ — объект был вызван хотя бы один раз.

__assert_called_with()__ — объект был вызван с таким-то значением.

__assert_called_once_with()__ — объект был вызван хотя бы один раз с таким-то значением.

Mock-объекты удобны еще тем, что позволяют заранее моделировать ответ на вызов того или иного метода, устанавливать значение переменной. Это означает, что в примере с базой данных в тестах, в методе setUp() можно создать Mock, эмулирующий подключение к базе данных, выполнение запросов к ней и ответов. Это в итоге позволит абстрагировать написание тестов от необходимости подключения к БД, а также от конкретных особенностей различных баз данных.

In [21]:
mock = Mock(name="Real Python Mock")
mock.one

<Mock name='Real Python Mock.one' id='140305958605456'>

In [22]:
mock = Mock()
mock.name = "Real Python Mock"
mock.name

'Real Python Mock'

Тут Mock-объект mock.name заменяется на строку и теперь при вызове mock.name в ответе будет именно строка.

Рассмотрим пример применения Mock-объекта при написании тестов:

In [ ]:
import unittest
from unittest.mock import Mock

class SomeTextCorrector:
  def __init__(self, items, id=None):
    self.items = items

  def correct(self, text_object):
    temp_str = text_object.original_text
    for substring in self.items["thrash_substrings"]:
      if substring in temp_str:
        temp_str = temp_str.replace(substring, "")
    text_object.original_text = temp_str.strip()
    return text_object

class TestSomeTextCorrector(unittest.TestCase):
  @classmethod
  def setUpClass(cls):
    items = {
        'thrash_substrings': ["спам", "кредит"]
    }
    cls.filter.SomeTextCorrector(items)

    def test1(self):
      text_object = Mock()
      text_object.original_text = "спам всегда приходит по возможности взять кредит"
      result = TestSomeTextCorrector.correct(text_object)
      self.assertEqual('всегда приходит по возможности взять', result.originl_text)

if __name__ == '__main__':
  unittest.main()

Тут создается класс SomeTextCorrector, который с помощью метода correct() удаляет лишние строки из original_text объекта text_object.

Так как во время написания теста мы не знаем о том, что из себя представляет класс, объектом которого является text_object, мы используем класс Mock для создания заглушки этого класса. Затем создаем переменную original_text и присваиваем ей строку со словами «спам» «кредит», чтобы метод correct() правильно отработал. Вызываем метод correct() и передаем Mock-объект и проверяем результат.